In [1]:
# RMatOfBunge...?

from numpy import *

def get_R_o(EAngles):

    # returns rotation matrix
            
    R_o = zeros((3,3),dtype=double)
    
    euler_phi1 = EAngles[0]
    euler_Phi = EAngles[1]
    euler_phi2 = EAngles[2]
    
    R_o[0,0]= cos(euler_phi1)*cos(euler_phi2) - sin(euler_phi1)*sin(euler_phi2)*cos(euler_Phi)
    R_o[1,0]= -cos(euler_phi1)*sin(euler_phi2) - sin(euler_phi1)*cos(euler_phi2)*cos(euler_Phi)
    R_o[2,0]= sin(euler_phi1)*sin(euler_Phi)
    R_o[0,1]= sin(euler_phi1)*cos(euler_phi2) + cos(euler_phi1)*sin(euler_phi2)*cos(euler_Phi)
    R_o[1,1]= -sin(euler_phi1)*sin(euler_phi2) + cos(euler_phi1)*cos(euler_phi2)*cos(euler_Phi) 
    R_o[2,1]= -cos(euler_phi1)*sin(euler_Phi)
    R_o[0,2]= sin(euler_phi2)*sin(euler_Phi)  
    R_o[1,2]= cos(euler_phi2)*sin(euler_Phi)
    R_o[2,2]= cos(euler_Phi) 

    return asmatrix(R_o)

In [2]:
import numpy as np
from math import sqrt

def bcrossn():
    b = np.array([[0, 1, -1],
                 [1, 0, -1],
                 [1, -1, 0],
                 [0, -1, 1],
                 [1, 0, 1],
                 [1, 1, 0],
                 [0, 1, 1],
                 [-1, 0, 1],
                 [-1, -1, 0],
                 [0, -1, -1],
                 [-1, 0, -1],
                 [-1, 1, 0]])
    
    n = np.array([[1, 1, 1],
                 [1, 1, 1],
                 [1, 1, 1],
                 [-1, 1, 1],
                 [-1, 1, 1],
                 [-1, 1, 1],
                 [1, -1, 1],
                 [1, -1, 1],
                 [1, -1, 1],
                 [1, 1, -1],
                 [1, 1, -1],
                 [1, 1, -1]])
    
    m = np.empty((12,3,3))
    q = np.empty((12,3,3))
    for i in range(12):
        Schmid = ((1 / sqrt(6)) * b[i,:].reshape((3,1)) * n[i,:])
        m[i,:,:] = .5 * (Schmid + Schmid.T)
        q[i,:,:] = .5 * (Schmid - Schmid.T)
    return m,q

In [3]:
def vertex_stress(i):
    v = np.zeros((28,6))
    v[0] = [1, -1, 0, 0, 0, 0]
    v[1] = [0, 1, -1, 0, 0, 0]
    v[2] = [-1, 0, 1, 0, 0, 0]
    v[3] = [0, 0, 0, 1, 0, 0]
    v[4] = [0, 0, 0, 0, 1, 0]
    v[5] = [0, 0, 0, 0, 0, 1]
    v[6] = [.5, -1, .5, 0, .5, 0]
    v[7] = [.5, -1, .5, 0, -.5, 0]
    v[8] = [-1, .5, .5, .5, 0, 0]
    v[9]= [-1, .5, .5, -.5, 0, 0]
    v[10]= [.5, .5, -1, 0, 0, .5]
    v[11]= [.5, .5, -1, 0, 0, -.5]
    v[12]= [.5, 0, -.5, .5, 0, .5]
    v[13]= [.5, 0, -.5, -.5, 0, .5]
    v[14]= [.5, 0, -.5, .5, 0, -.5]
    v[15]= [.5, 0, -.5, -.5, 0, -.5]
    v[16]= [0, -.5, .5, 0, .5, .5]
    v[17]= [0, -.5, .5, 0, -.5, .5]
    v[18]= [0, -.5, .5, 0, .5, -.5]
    v[19]= [0, -.5, .5, 0, -.5, -.5]
    v[20]= [-.5, .5, 0, .5, .5, 0]
    v[21]= [-.5, .5, 0, -.5, .5, 0]
    v[22]= [-.5, .5, 0, .5, -.5, 0]
    v[23]= [-.5, .5, 0, -.5, -.5, 0]
    v[24]= [0, 0, 0, .5, .5, .5]
    v[25]= [0, 0, 0, .5, -.5, .5]
    v[26]= [0, 0, 0, -.5, .5, .5]
    v[27]= [0, 0, 0, .5, .5, .5]

    return sqrt(6) * v[i]

In [4]:
def vect2dev_tensor(V):
    DT = np.zeros((3,3))
    DT[0,0] = (V[2] - V[1]) / 3
    DT[1,1] = (V[0] - V[2]) / 3
    DT[2,2] = (V[1] - V[0]) / 3
    DT[0,1] = V[5]
    DT[1,0] = V[5]
    DT[0,2] = V[4]
    DT[2,0] = V[4]
    DT[1,2] = V[3]
    DT[2,1] = V[3]
    return DT

In [5]:
def BHmodel(de_c, g, i):
    dW = []
    for j in range(28):
        vs = vertex_stress(j)
        A = vs[0]
        B = vs[1]
        C = vs[2]
        F = vs[3]
        G = vs[4]
        H = vs[5]

        dW.append(-B * de_c[0,0] + A * de_c[1,1] + 2 * F * de_c[2,1] + 2 * G * de_c[2,0] + 2 * H * de_c[0,1])
    
#     print dW
    maxdW = 0
    index = -1
    for i in range(len(dW)):
        if abs(dW[i]) > maxdW:
            maxdW = abs(dW[i])
            index = i
#     print maxdW
#     print index
    if dW[index] < 0:
        S_c = -1 * vertex_stress(index)
    else:
        S_c = vertex_stress(index)

    Stensor_c = vect2dev_tensor(S_c)
    return g.T * Stensor_c * g

In [6]:
def get_taylor(phi1,Phi,phi2):
    Bunge = np.array((phi1, Phi, phi2)).T
    
    ###################################
    de = np.array([[-.5,  0,   0],
                  [0, -.5,  0],
                  [0,  0,  1]])
    ###################################
    
    m, q = bcrossn()
#     Nc = len(Euler1)
    S = np.zeros((3,3))
    Sdotde = 0
    dedotde = 0
    eVM = 0
#     angle = np.zeros((Nc*Nc,1))
#     a = 11
    g = get_R_o(Bunge).T # I hope this works right, can't test it yet
#     print "Euler Angles (radians): ", Bunge[a], "\n"
#     print "RMatOfBunge/R_o: "
#     print g

    # print de
    de_c = g * de * g.T
    # print de_c

    Sc = BHmodel(de_c, g, 0)
    # print Sc
    Sdotde = 0
    dedotde = 0
    eVM = 0

    for k in range(3):
        for l in range(3):
            Sdotde += Sc[k,l] * de[k,l]
            dedotde += de[k,l] * de[k,l]

    eVM = sqrt((2.0 / 3.0) * dedotde)
    M = Sdotde / eVM
    return M

In [7]:
# import sqlite3

# def save_taylor(dbpath):
#     conn = sqlite3.connect(dbpath)
#     natee=0
#     cursor = conn.execute('select distinct short_grain_id,phi1,Phi,phi2 from points;')
#     for row in cursor:
#         short_grain_id = row[0]
#         phi1 = row[1]
#         Phi = row[2]
#         phi2 = row[3]
#         val = get_taylor(phi1, Phi, phi2)
#         if natee < 10:
#             print val,phi1,Phi,phi2
#         natee += 1
#     conn.close()

In [8]:
import sqlite3

def alter_tf(dbpath):
    conn = sqlite3.connect(dbpath)
    for r in range(6,43,2):#(6,61,3):
        for i in range(1,11):
            conn.execute('alter table data_%d'%r+' add column tf_%d float'%i)
        
        for i in range(1,11):
            for j in range(i + 1, 11):
                conn.execute('alter table data_%d'%r+' add column tf_%dto%d float'%(i,j))
    conn.close()
    print 'done'

In [9]:
import sqlite3
import math

def save_tf_data(dbpath):

    conn = sqlite3.connect(dbpath)
#     x=0
    for r in range(6,43,2):#(6,61,3):
    
        cursor = conn.execute('select crack_id, arr_index, num_xtal, ' +
            'phi1_1, phi1_2, phi1_3, phi1_4, phi1_5, phi1_6, phi1_7, phi1_8, phi1_9, phi1_10, ' +
            'Phi_1, Phi_2, Phi_3, Phi_4, Phi_5, Phi_6, Phi_7, Phi_8, Phi_9, Phi_10, ' +
            'phi2_1, phi2_2, phi2_3, phi2_4, phi2_5, phi2_6, phi2_7, phi2_8, phi2_9, phi2_10 from data_%d'%r)

        for row in cursor:
            crack_id=row[0]
            arr_index=row[1]
            num_xtal=row[2]
            
            phi1s = []
            Phis = []
            phi2s = []
            
            for i in range(3,13):
                phi1s.append(row[i])
                Phis.append(row[i+10])
                phi2s.append(row[i+20])
            
            if num_xtal > 1:
                if num_xtal > 10: # We don't care about more than 10 grains
                    num_xtal = 10
                
                tfs = []
                for i in range(num_xtal):
                    tfs.append(get_taylor(phi1s[i],Phis[i],phi2s[i]))
                    conn.execute('update data_%d set tf_%d=%f where crack_id=%d and arr_index=%d' \
                                %(r,i+1,tfs[i],crack_id,arr_index))
                
                for i in range(num_xtal):
                    for j in range(i + 1, num_xtal):
                        tf_diff = abs(tfs[i] - tfs[j])
                        conn.execute('update data_%d set tf_%dto%d=%f where crack_id=%d and arr_index=%d' \
                                 %(r,i+1,j+1,tf_diff,crack_id,arr_index))
#             x+=1
        print r
    conn.commit()
    conn.close()
    print 'done'

In [10]:
# Sets misorientation and taylor factor to null if the grains don't touch in the neighborhood. Touching diagonally 
# does not count.
def delete_if_no_boundary(dbpath):
    conn = sqlite3.connect(dbpath)
    x=0
    for r in range(6,43,2):
        cursor = conn.execute('select crack_id,arr_index,num_xtal,grain_id_1,grain_id_2,grain_id_3,grain_id_4,' +
                             'grain_id_5,grain_id_6,grain_id_7,grain_id_8,grain_id_9,grain_id_10 from data_%d'%r)
        for row in cursor:
            crack_id = row[0]
            arr_index = row[1]
            num_xtal = row[2]
            grains = []
            if num_xtal > 10:
                num_xtal=10
            for i in range(num_xtal):
                grains.append(row[i + 3])
            
            for i in range(num_xtal):
                    for j in range(i + 1, num_xtal):
                        cursor2 = conn.execute('select * from boundaries where grain_id1=%d and grain_id2=%d' \
                                               %(grains[i],grains[j]))
                        exists = False
                        for row2 in cursor2:
                            exists = True

                        if exists is False:
                            conn.execute('update data_%r set mis_%dto%d=null,tf_%dto%d=null where'%(r,i+1,j+1,i+1,j+1)+
                                         ' crack_id=%d and arr_index=%d'%(crack_id,arr_index))
    conn.commit()
    conn.close()
    print 'done'

In [191]:
# alter_tf('../../big-data-smaller-r/base.db')

In [192]:
# save_tf_data('../../big-data-smaller-r/base.db')

In [11]:
# delete_if_no_boundary('../../big-data-smaller-r/base.db')

6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
